In [5]:
import pandas as pd
import nltk
import torch
import tensorflow as tf
import numpy as np
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
import simpletransformers
import seaborn as sns
import wandb
import tarfile

In [2]:
tarf = tarfile.open('gpt_personachat_cache.tar.gz')
tarf.extractall()
tarf.close()

In [7]:
from simpletransformers.conv_ai import ConvAIModel


train_args = {
    "overwrite_output_dir": True,
    "reprocess_input_data": True
}

# Create a ConvAIModel
model = ConvAIModel("gpt", "gpt_personachat_cache", use_cuda=False, args=train_args)

ftfy or spacy is not installed using BERT BasicTokenizer instead of SpaCy & ftfy.


In [11]:
model.train_model("data/train.json")

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/transformers/modeling_openai.py:690: FutureWarning: The `lm_labels` argument is deprecated and will be removed in a future version, use `labels` instead.
  FutureWarning,


Running loss: 4.129050



## BERT

In [ ]:
model_args = ClassificationArgs(num_train_epochs=1, overwrite_output_dir=True, manual_seed=42)
model = ClassificationModel(model_type='distilbert', model_name='distilbert-base-cased', 
                            use_cuda=False, num_labels=2, args=model_args)

In [ ]:
%%time
model.train_model(train_df)

In [ ]:
help(simpletransformers.classification.)

## Sequence model

In [13]:
labels = df_train['target']

In [14]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(
    num_words=10000, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', lower=True,
    split=' ', char_level=False, oov_token=None, document_count=0)
tokenizer.fit_on_texts(df_train['text'])
train_seq = np.array(tokenizer.texts_to_sequences(df_train['text']))
train_vector = tf.keras.preprocessing.sequence.pad_sequences(train_seq, padding='post')
train_vector = np.reshape(train_vector, (train_vector.shape[0], 1, train_vector.shape[1])).astype(float)
train_y = np.array(labels).astype(float)

In [15]:
tr_x, va_x, tr_y, va_y = train_test_split(train_vector, train_y, stratify=train_y, random_state=342)

In [40]:
model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(128, input_shape = tr_x.shape[1:], return_sequences=True),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [41]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.RMSprop(1e-4),
              metrics=['accuracy'])

In [42]:
history = model.fit(tr_x, tr_y, epochs=70, validation_data=(va_x, va_y), verbose=1)

Epoch 1/70
179/179 [==============================] - 2s 11ms/step - loss: 0.7500 - accuracy: 0.5658 - val_loss: 0.7389 - val_accuracy: 0.5704
Epoch 2/70
179/179 [==============================] - 1s 4ms/step - loss: 0.7252 - accuracy: 0.5703 - val_loss: 0.7147 - val_accuracy: 0.5704
Epoch 3/70
179/179 [==============================] - 1s 5ms/step - loss: 0.7041 - accuracy: 0.5703 - val_loss: 0.7003 - val_accuracy: 0.5704
Epoch 4/70
179/179 [==============================] - 1s 5ms/step - loss: 0.6957 - accuracy: 0.5703 - val_loss: 0.6957 - val_accuracy: 0.5704
Epoch 5/70
179/179 [==============================] - 1s 5ms/step - loss: 0.6932 - accuracy: 0.5703 - val_loss: 0.6943 - val_accuracy: 0.5704
Epoch 6/70
179/179 [==============================] - 1s 4ms/step - loss: 0.6923 - accuracy: 0.5702 - val_loss: 0.6938 - val_accuracy: 0.5709
Epoch 7/70
179/179 [==============================] - 1s 5ms/step - loss: 0.6916 - accuracy: 0.5716 - val_loss: 0.6938 - val_accuracy: 0.5735
Epoch

In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 1, 128)            82432     
_________________________________________________________________
conv1d (Conv1D)              (None, 1, 64)             8256      
_________________________________________________________________
dense (Dense)                (None, 1, 256)            16640     
_________________________________________________________________
flatten (Flatten)            (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 257       
Total params: 107,585
Trainable params: 107,585
Non-trainable params: 0
_________________________________________________________________
